# Introduction to blockchain HW 1

## Problem 1 (10 points)

Assume that at the end of the term an instructor uses an RSA Public Key Cryptosystem to sign the grades sent to students and to the Education Office. He signs each grade with his private key and transmits both the grade and the signature in a secure manner encrypted with a public key. After you decrypt the message you use his public key to decipher the signature and verify the grade. Assume the cryptosystem has the following features: n=55, e=33 and that the grading system features integer grades from 0 to 10. You have received the grade-signature pairs 8||13 and 6||41

1.Verify that the grades you received are indeed originating from the instructor showing just intermediate results of your calculation (2 points)

To verify, we need to check this
$$13^{33}\  mod\  55 =^? 8$$
$$41^{33}\  mod\  55 =^? 6$$

If we calculate it...
We get $13^{33}\  mod\  55 = 8$ and $41^{33}\  mod\  55 = 6$.

It means that signatures are correct

2.Given the information above only can you fabricate the instructors’ signature for another grade? If yes, for which grade and how? (3 points)

Any grade. See the code below

In [61]:
%%time
s = 0
while s**33 % 55 != 10:
    s += 1
print(s)

10
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.13 ms


It's even possible to find d value which was used for signing and generate any possible mark and signature faster

So we know that
$$(2^3)^d = 13\ mod\ 55$$
$$2^d 3^d = 41\ mod\ 55$$
This can be transformed to
$$2^d(2^{2d}+3^d)\ mod\ 55 = 54$$
Writing the simple cycle above we can get d value. So now we can easily generate new signatures.

In [64]:
%%time
for d in range(1,100):
    if (2**d)*(2**(2*d)+3**d) % 55 == 54:
        print(d)
        break

17
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.95 ms


For example we need 9 grade
$$9^{17}=(s_{new}^{33})^{17} = s_{new}\ mod\ 55$$
Calculating $9^{17}\ mod\ 55$ we get $s_{new} = 4$. Using the name of our teacher we can send 9||4 pair.

In [1]:
9**17 % 55

4

##### 3.What would you advise the instructor to do so as to prevent such fabrication? (2 points)

It's better to use bigger (way more bigger) numbers $N$ and $e$ to make this computational task not so easy

4.Discuss what can go wrong in case the instructor sends a grade secretly by first enciphering it with a student’s public key, the signing it with his private key. Provide an example (3 points)

For example. The grade is 6 and after enciphering this mark with student's public key we get value 51.

In [8]:
6**33 % 55

51

Next, let's create the signature with $d=17$. 

In [9]:
s = 51**17 % 55
print(s)

6


So we've got the signature 6. And the pair after decryption by student will be 6 || 6

The signature check will be true.

Also, each pair will look like this 1 || 1, 2 || 2, 3 || 3 and so on, because
$$\forall x \in Z_n,\ x^{ed}\ mod\ n = x$$
$$mark^{33}\ mod\ 55 = m$$
$$m^{17}\ mod\ 55 = s^{17 \cdot 33}\ mod\ 55 = m = s$$

So, it would be easy to break these system

## Problem 2 (10 points)

El-Gamal is videly used cryptographic standart. In this task you will implement El-Gamal encryption scheme using Python

1.Implement function for generating keys. The function must generate big random prime number (problem of generating big prime numbers was discussed during seminar after lecture 3). (2 points)

In [41]:
from math import sqrt
from random import getrandbits, randrange

def gcd(a, b): 
    if (a == 0): 
        return b 
    return gcd(b % a, a) 

def phi(n):   
    result = 1
    for i in range(2, n): 
        if (gcd(i, n) == 1): 
            result+=1
    return result 

def powmod(base, degree, module):
    degree = bin(degree)[2:]
    r = 1
 
    for i in range(len(degree) - 1, -1, -1):
        r = (r * base ** int(degree[i])) % module
        base = (base ** 2) % module
 
    return r
 
    
def get_g_2(p):
    fact = []
    # phi_value = phi(p)
    phi_value = p - 1
    n = phi_value
    i = 2
    while i*i <= n:
        if n % i == 0:
            fact.append(i)
            while n % i == 0:
                n /= i
        i += 1
                        
    if n > 1:
        fact.append(n)
    
    res = 2
    while res <= p:
        if gcd(res, n) == 1:
            ok = True
            i = 0
            while (i < len(fact)) and ok:
                ok &= (powmod(int(res), int(phi_value / fact[i]), int(p)) != 1)
                i += 1
            if ok:
                return res
        res += 1
    return -1;

def get_g(modulo):
    g = 1
    phi_value = phi(modulo)
    while True:
        for i in range(1, phi_value + 1):
            if g**i % modulo == 1:
                if i == phi_value:
                    return g
                else:
                    break
        g += 1

def prime_test(n, k=128):
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def generate_prime_candidate(length):
    p = getrandbits(length)
    p |= (1 << length - 1) | 1 # big and not even
    return p

def generate_prime_number(length=1024):
    p = 4
    while not prime_test(p, 128):
        p = generate_prime_candidate(length)
    return p

2.Implement functions that realize the encryption and decryption functions. (2 points)

In [42]:
from random import randint

def encrypt(mes, prime, public, g): # y is public key
    K = randint(2, prime-2)
    return powmod(g, K, prime), (powmod(public,K, prime)*mes) % prime # a and b


def decrypt(private, prime, a, b):
    return b*powmod(a, prime-1-private, prime) % prime

3.Test your functions on random values and show that your realization works correctly (1 point)

In [59]:
%%time
prime_number = generate_prime_number(1024)
print("New prime number {}".format(prime_number))

New prime number 91232401780399592906967034025840779293640230858008585237595362343192089341155435194883646296258249272668847154312981079314746452631367862490010550053529263413209523184563644995153016772026532218757711882800053603932293303189686887536531959079524020731447286362224344786067270475444031526551861537609634871943
CPU times: user 1.66 s, sys: 4 ms, total: 1.66 s
Wall time: 1.66 s


In [60]:
g = get_g_2(prime_number)

In [62]:
private_key = randint(2, prime_number-2)

In [63]:
public_key = powmod(g, private_key, prime_number)

In [64]:
message = randint(1,prime_number)
print("Original message {}".format(message))
a,b = encrypt(message, prime_number, public_key, g)
print("After enctyption {} {}".format(a,b))
decrypted = decrypt(private_key, prime_number, a, b)
print("Decrypted result {}".format(decrypted))
print("The process is {}".format(message == decrypted))

Original message 48539575452135382874411192673377751191733672100013635659068525818200074173891712487499894218858415209181288585431633509326444940603037068479554801571215021652533908242283390868119939334349234619044663780561452778047146622761646031271919050413076331257857022423682459228376671764805830146875733402248048661313
After enctyption 30020476051753792500002707213007879596487269038025949322244867954445693946518586981748313395816035330939835288993453924767868597867446915945246181703557966871583425666238517621271079910383429710460343833861371024358344705130811208693464221444798737312279146416577266266753940370590221728232755832889651957462 11814762548971407916823549565348814460561450310300354254894172077382192182665759187288104346847666411902561090404177410636079642493978535078472755736811112366204376351721729944338167439206468996516987220527625315754703071155894842695129776707376494414708920183358686522604131593649793379812348320332710820008
Decrypted result 4853957545213538287441

4.Implement functions that realize creation and verification of digital signature (2 points)

In [92]:
def myhash(val):
    return val


def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = egcd(b % a, a)
        return (g, y - (b // a) * x, x)
 

def find_inverse(b, n):
    g, x, _ = egcd(b, n)
    if g == 1:
        return x % n


def create_signature(message, prime, private_key):
    hsum = myhash(message)
    k = prime - 1
    while gcd(k, prime-1) != 1:
        k = randint(2, prime-2)
    r = powmod(g, k, prime)
    s = (hsum-private_key*r)*find_inverse(k, prime - 1) % (prime - 1)
    return r,s


def verify_signature(message, p,g,y,r,s): # message расшифрованное уже,(p,g,y) открытый ключ, r,s - подпись 
    if not (((r > 0) and (r < p)) and ((s > 0) and (s < p-1))):
        return False
    else:
        m = myhash(message)
        if (powmod(y, r, p)*powmod(r, s, p) % p) == powmod(g, m, p):
            return True
        else:
            return False

5.Test your functions on random values and show that your algorithm works correctly (1 point)

In [93]:
r,s = create_signature(message, prime_number, private_key)
print(r,s)
print(verify_signature(message, prime_number, g, public_key, r, s))

10550172429920474504965971602654993122711029789027853163629038243724030110429433900764048943862885088304063202484390549938869429742755647983151465935707559013207564723414028139891182526186277758144100428017644900466875438220177077931424317967952111622677509033942251612438190764733264157683247247910968367451 49497411133002414594975372912788904235068543382240511229326517382303941454929273180482433114540998196107549453502080985260938034674821870308757822156112879152864645208731284221908389470629746113867200222420123078261462366120609560683919736144035965613377927291191430365489913500252297728351811058786530907821
True


## Problem 3 (15 points)

1.Implement SHA256 (https://en.wikipedia.org/wiki/SHA-2) hashing algorithm using the pseudo-code below. Note the great increase in mixing between bits of the w[16..63] words compared to SHA-1 (10 points)

In [555]:
INT_BYTES = 4
BITS_IN_INT = 8

def unsigned_not(s): # integer in bits string
    res = ''
    for elem in s:
        if elem == '1':
            res += '0'
        if elem == '0':
            res += '1'
            
    return res
            
def bit_str_array(s):
    res = []
    for symbol in s:
        not_prepended = "{0:b}".format(ord(symbol))
        prepended = not_prepended
        while len(prepended) != INT_BYTES * BITS_IN_INT:
            prepended = '0' + prepended
        res.append(prepended)
    if res:
        return res
    else:
        return [''.join(['0' for i in range(INT_BYTES * BITS_IN_INT)])]


INT_BITS = INT_BYTES * BITS_IN_INT


def rightRotate(n, d): 
    return (n >> d)|(n << (INT_BITS - d)) & 0xFFFFFFFF

    
def h(message):    
    h0 = 0x6a09e667
    h1 = 0xbb67ae85
    h2 = 0x3c6ef372
    h3 = 0xa54ff53a
    h4 = 0x510e527f
    h5 = 0x9b05688c
    h6 = 0x1f83d9ab
    h7 = 0x5be0cd19

    k = [
       0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
       0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
       0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
       0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
       0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
       0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
       0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
       0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

    number_of_bits_in_message = len(message) * INT_BYTES * BITS_IN_INT
    bitarr = bit_str_array(message)
    modified_mes = ''
    for elem in bitarr:
        modified_mes += elem

    modified_mes += '1'
    
    K = 0
    while (number_of_bits_in_message + 1 + K) % 512 != 448:
        K += 1

    modified_mes += ''.join(['0' for i in range(K)])

    mes_len_in_64_bits = '{0:b}'.format(len(message))
    while len(mes_len_in_64_bits) != 64:
        mes_len_in_64_bits = '0' + mes_len_in_64_bits

    modified_mes += mes_len_in_64_bits
    for i in range(0, len(modified_mes), 512):
        modified_message_slice = modified_mes[i:i+512]
        w = []
        
        for z in range(0, len(modified_message_slice), 32):
            w.append(modified_message_slice[z:z+32])
        
        for idx in range(len(w)):
            w[idx] = int(w[idx], 2)
        
        while len(w) != 64:
            w.append(0)

        for j in range(16, 64):
            s0 = (rightRotate(w[j-15], 7) ^ rightRotate(w[j-15], 18) ^ (w[j-15] >> 3))
            s1 = (rightRotate(w[j-2], 17) ^ rightRotate(w[j-2], 19) ^ (w[j-2] >> 10))
            w[j] = (w[j-16] + s0 + w[j-7] + s1) & 0xffffffff 

        a = h0
        b = h1
        c = h2
        d = h3
        e = h4
        f = h5
        g = h6
        h = h7

        for z in range(64):
            s1 = rightRotate(e, 6) ^ rightRotate(e, 11) ^ rightRotate(e, 25)
            ch = (e & f) ^ (int(unsigned_not('{0:b}'.format(e)),2) & g)
            temp1 = (h + s1 + ch + k[z] + w[z]) & 0xffffffff
            s0 = rightRotate(a, 2) ^ rightRotate(a, 13) ^ rightRotate(a, 22)
            maj = (a & b) ^ (a & c) ^ (b & c)
            temp2 = (s0 + maj) & 0xffffffff
            
            h = g
            g = f
            f = e
            e = (d + temp1) & 0xffffffff
            d = c
            c = b
            b = a
            a = (temp1 + temp2) & 0xffffffff

        h0 = (h0 + a) & 0xffffffff 
        h1 = (h1 + b) & 0xffffffff 
        h2 = (h2 + c) & 0xffffffff 
        h3 = (h3 + d) & 0xffffffff 
        h4 = (h4 + e) & 0xffffffff 
        h5 = (h5 + f) & 0xffffffff 
        h6 = (h6 + g) & 0xffffffff 
        h7 = (h7 + h) & 0xffffffff 

    hbits = [
        '{0:b}'.format(h0),
        '{0:b}'.format(h1),
        '{0:b}'.format(h2),
        '{0:b}'.format(h3),
        '{0:b}'.format(h4),
        '{0:b}'.format(h5),
        '{0:b}'.format(h6),
        '{0:b}'.format(h7)
    ]
    adjusted = []
    for elem in hbits:
        while len(elem) != 32:
            elem = '0' + elem
        adjusted.append(elem)

    digest = ''.join(adjusted)
    integer_digest = int(digest, 2)
    return digest, hex(integer_digest)

2.Calculate hashes of the texts below (1 point)

In [556]:
string_small = 'This is a very small string with a few characters.'
string_larger = 'This is a larger string that contains more characters.'
string_big = 'This is a larger string that contains more characters. This demonstrates that no matter how big the input stream is, the generated hash is the same size (but of course, not the same value). If two files have a different hash, they surely contain different data.'
string_empty = ''

In [557]:
print(h(string_small)[1])
print(h(string_larger)[1])
print(h(string_big)[1])
print(h(string_empty)[1])

0x6365f027ebc0a275cc0a6b0f2fe7ad76cd50276213443a13819945ac2de15cca
0xa34daee8eecb351a18542d4f328ef24cde4f7a7d35afcea672ab1e314def51f8
0x5c78960e579e64ea656ab52e43198cbdfbebcc311afb010fffcad724e36806a0
0x798b854dd5cba1a92690586658c2cf031633e800d02c718f9f7490a2b05095ee


3.What is a bit length of each hash? (1 point)

256

4.What is the bitwise distance between them? What is bitwise distance between their hashes? (1 point)

In [558]:
from itertools import combinations



for comb in combinations([('string_small', ''.join(bit_str_array(string_small))), 
                            ('string_larger', ''.join(bit_str_array(string_larger))),
                            ('string_big', ''.join(bit_str_array(string_big))),
                            ('string_empty', ''.join(bit_str_array(string_empty)))],2):
    dist = 0
    maxlen = max([len(comb[0][1]), len(comb[1][1])])
    
    a = comb[0][1]
    while len(a) != maxlen:
        a += '0'
    b = comb[1][1]
    while len(b) != maxlen:
        b += '0'
        
    for idx in range(maxlen):
        if a[idx] != b[idx]:
            dist += 1
            
    print("{}|{}. Distance {}".format(comb[0][0], comb[1][0], dist))


for comb in combinations([('string_small_hash', h(string_small)[0]), 
                            ('string_larger_hash', h(string_larger)[0]),
                            ('string_big_hash', h(string_big)[0]),
                            ('string_empty_hash', h(string_empty)[0])],2):
    dist = 0
    for idx in range(256):
        if comb[0][1][idx] != comb[1][1][idx]:
            dist += 1
    print("{}|{}. Distance {}".format(comb[0][0], comb[1][0], dist))

string_small|string_larger. Distance 125
string_small|string_big. Distance 858
string_small|string_empty. Distance 178
string_larger|string_big. Distance 733
string_larger|string_empty. Distance 197
string_big|string_empty. Distance 930
string_small_hash|string_larger_hash. Distance 129
string_small_hash|string_big_hash. Distance 137
string_small_hash|string_empty_hash. Distance 134
string_larger_hash|string_big_hash. Distance 136
string_larger_hash|string_empty_hash. Distance 139
string_big_hash|string_empty_hash. Distance 123


5.Typically use apply hash function to our passwords and texts that we want to digitally sign. Implement digital signature of hashed string using El-Gamal digital signature. Compare the digital signature of pain text and hashed text. (2 points)

In [559]:
r,s = create_signature(int(''.join(bit_str_array(string_big)),2), prime_number, private_key)
print(r,s)
print(verify_signature(int(''.join(bit_str_array(string_big)),2), prime_number, g, public_key, r, s))

25050570410845749308776732517010144560682448931608645733600420423327244023779254584695598915996292572772709658076184400720843556307970819431916146843943948840730286491167028862583416984251678245119051558239972844383307002640275758498817054285016216311013334414522690589048903549780214193887034167406939811856 33819612555556953076119488869286248249044807956678348141108950193810926584564639799303304845801050048295136059484651584095624537193889657695083438550036558510052480156359605230316148889414148638102548352294976091650597796120953857553161340661842398197719177001068218269479020813855273406571394225918695638782
True


In [560]:
r,s = create_signature(int(h(''.join(bit_str_array(string_big)))[1],16), prime_number, private_key)
print(r,s)
print(verify_signature(int(h(''.join(bit_str_array(string_big)))[1],16), prime_number, g, public_key, r, s))

89985447853231801004433189094994703360864123131307361244230641073640730592543753865201038116181498509509069152262896011354943242306728177743179836333282597924900464640508290869229603234063699144563639800321414635456806053639062662388065467342756468089938804074743921300543396094130746899347709683679611992007 48912606636108024748712289400156161694440221071422497716671808974873528790167346413833795026204530836676771218869780697024658593808285541614840749960123521197981514821078529010496227592688768290521455639320169789152328046450306462836898774113342206429804427742280606925407545595981896270567457120615003396646
True


It seems like they are different. Suppose this is good

## Problem 4 (15 points)

Merkle hash trees play an important role in forming transaction blocks in blockchain. In this assignment we ask you to plot your own Merkle hash tree and check its' properties. Below we provide you with some code fragment what you can use in your assignment

In [100]:
import matplotlib
import networkx as nx
%matplotlib qt5
from networkx import balanced_tree, draw_networkx, draw

Let us plot graph basis for Merkle hash tree

In [188]:
G = nx.Graph()
positions = {}
coordinates = [
    [0, 4],
    [-2, 3],
    [2, 3],
    [-3, 2],
    [-1, 2],
    [1, 2],
    [3, 2],
    [-3, 1],
    [-1, 1],
    [1, 1],
    [3, 1]
]
parents = [0, 0, 0, 1, 1, 2, 2, 3, 4, 5, 6]
for index in range(11):
    G.add_node(index)
    G.add_edge(index, parents[index])
    positions[index] = coordinates[index]
nx.draw(G, positions, node_size = 1000)
labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: 'tx1',
    8: 'tx2',
    9: 'tx3',
    10: 'tx4',
}
nx.draw_networkx_labels(G, positions, labels = labels)

/home/dmitriy/MyEnv/other/skoltech/intro-to-blockchain/env/lib/python3.5/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


{0: Text(0, 4, '0'),
 1: Text(-2, 3, '1'),
 2: Text(2, 3, '2'),
 3: Text(-3, 2, '3'),
 4: Text(-1, 2, '4'),
 5: Text(1, 2, '5'),
 6: Text(3, 2, '6'),
 7: Text(-3, 1, 'tx1'),
 8: Text(-1, 1, 'tx2'),
 9: Text(1, 1, 'tx3'),
 10: Text(3, 1, 'tx4')}

In Bitcoin double sha256 hash scheme is used. Here is an example.

In [189]:
import hashlib


first_hash = hashlib.sha256(b"hello") # "b" stands for binary representation
second_hash = hashlib.sha256()
print('First hash represented as a hexadecimal number:', first_hash.hexdigest())
second_hash.update(first_hash.digest())
print('Second hash represented as a hexadecimal number:', second_hash.hexdigest())

First hash represented as a hexadecimal number: 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
Second hash represented as a hexadecimal number: 9595c9df90075148eb06860365df33584b75bff782a510c6cd4883a419833d50


Now we can easily change vertices' labels to hashes of corresponding messages and plot new graph

In [190]:
def bitcoin_hash(s):
    s = str(s).encode('utf-8')
    return hashlib.sha256(hashlib.sha256(s).digest()).hexdigest()

In [191]:
labels[3] = bitcoin_hash(b"tx1")

# and plot the graph again

nx.draw(G, positions, node_size = 1000)
nx.draw_networkx_labels(G, positions, labels = labels, font_size = 8)
print(labels[3])

cce4324fe6e4db20ac482dbbe8ad672764cc8bcc303eed3b460212d096a90f03


1.Construct Merkle hash tree using previously constructed graph by finding corresponding SHA256 hashes on vertices (2 points). Plot obtained Merkle hash tree (1 point)

In [192]:
labels[4] = bitcoin_hash(b"tx2")
labels[5] = bitcoin_hash(b"tx3")
labels[6] = bitcoin_hash(b"tx4")

In [193]:
labels[1] = bitcoin_hash(labels[3] + labels[4])
labels[2] = bitcoin_hash(labels[5] + labels[6])
labels[0] = bitcoin_hash(labels[1] + labels[2])

In [194]:
nx.draw(G, positions, node_size = 1000)
nx.draw_networkx_labels(G, positions, labels = labels, font_size = 8)

{0: Text(0, 4, '917b01735e2cba808bd592d278124843a3036021fa1686513e13ebdfc5886cce'),
 1: Text(-2, 3, '5dbf5a1e980a4073ad313983e05c33ac7c8396de81d9d00e292d4db9b7c35e07'),
 2: Text(2, 3, '4852a5def31ddef20701d5c56ad58585efe5a3a22b1e46c2e8f5850d62c42e90'),
 3: Text(-3, 2, 'cce4324fe6e4db20ac482dbbe8ad672764cc8bcc303eed3b460212d096a90f03'),
 4: Text(-1, 2, 'a510a994957496a32c7fea49141b7adedfe205117142db675a68273fe3058dc9'),
 5: Text(1, 2, '9c9a6b8260f419729dfc86b1058c4f5881c699292af6183e05f95f51f49531f5'),
 6: Text(3, 2, '36d87c9b1bb7e42101cfc85ace37cce966af1d6906e4dc8ac48db12c51659e4a'),
 7: Text(-3, 1, 'tx1'),
 8: Text(-1, 1, 'tx2'),
 9: Text(1, 1, 'tx3'),
 10: Text(3, 1, 'tx4')}

2.Provide a proof of correctness of leaf tx2 (2 points). 

In [203]:
labels[0] == bitcoin_hash(bitcoin_hash(labels[3]+bitcoin_hash(b"tx2")) + labels[2])

True

3.Provide a proof of correctness for set of leafs (tx3-tx4) (2 points)

In [204]:
labels[0] == bitcoin_hash(labels[1] + bitcoin_hash(bitcoin_hash(b"tx3") + bitcoin_hash(b"tx4")))

True

4.Change the value on leaf tx1 and recompute corresponding hashes. Plot newly obtained Merkle hash tree (2 points)

In [210]:
labels[7] = 'tx1olol'
labels[3] = bitcoin_hash(b"tx1olol")
labels[4] = bitcoin_hash(b"tx2")
labels[5] = bitcoin_hash(b"tx3")
labels[6] = bitcoin_hash(b"tx4")

In [211]:
labels[1] = bitcoin_hash(labels[3] + labels[4])
labels[2] = bitcoin_hash(labels[5] + labels[6])
labels[0] = bitcoin_hash(labels[1] + labels[2])

In [212]:
nx.draw(G, positions, node_size = 1000)
nx.draw_networkx_labels(G, positions, labels = labels, font_size = 8)

{0: Text(0, 4, '6810a02cfec33772ee14dc30b094f16329e4c320e51fdc321fad9d2197137cdf'),
 1: Text(-2, 3, '6d97ba6d484990d1ed2516b85110457d980796c9d46ae2b563f10148b86d3b9a'),
 2: Text(2, 3, '4852a5def31ddef20701d5c56ad58585efe5a3a22b1e46c2e8f5850d62c42e90'),
 3: Text(-3, 2, 'f869ee683a57ed9bdf53973b121e4e4e698858ddfd611caffe78c995a68c3830'),
 4: Text(-1, 2, 'a510a994957496a32c7fea49141b7adedfe205117142db675a68273fe3058dc9'),
 5: Text(1, 2, '9c9a6b8260f419729dfc86b1058c4f5881c699292af6183e05f95f51f49531f5'),
 6: Text(3, 2, '36d87c9b1bb7e42101cfc85ace37cce966af1d6906e4dc8ac48db12c51659e4a'),
 7: Text(-3, 1, 'tx1olol'),
 8: Text(-1, 1, 'tx2'),
 9: Text(1, 1, 'tx3'),
 10: Text(3, 1, 'tx4')}

5.Nodes in Merkle hash trees may have arbitrary fanout. In previouse items we consider the case of fanout equals to two. But what will change if we set a fanout equals to three? Construct Merkle hash trees with fanout 3 to sign 9 values? Construct the hash tree with fanout 2 to sign the same set of values? Plot obtained trees (4 points) 

Merkle tree is balanced, so there'll be more leaves in the bottom. Hence we don't have 9 values (we have only 4), tx4 we'll be used for the hash calculation on the right side of the tree

In [217]:
G = nx.Graph()
positions = {}
coordinates = [
    [0, 4],
    [-3, 3],
    [0, 3],
    [3, 3],
    [-4, 2],
    [-3, 2],
    [-2, 2],
    [-1, 2],
    [0, 2],
    [1, 2],
    [2, 2],
    [3, 2],
    [4, 2],
    [-5, 1],
    [-4, 1],
    [-3, 1],
    [-2, 1],
    [-1, 1],
    [0, 1],
    [1, 1],
    [2, 1],
    [3, 1]
]
parents = [0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
for index in range(22):
    G.add_node(index)
    G.add_edge(index, parents[index])
    positions[index] = coordinates[index]
nx.draw(G, positions, node_size = 1000)
labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: '7',
    8: '8',
    9: '9',
    10: '10',
    11: '11',
    12: '12',
    13: 'tx1',
    14: 'tx2',
    15: 'tx3',
    16: 'tx4',
    17: 'tx5',
    18: 'tx6',
    19: 'tx7',
    20: 'tx8',
    21: 'tx9',
}
nx.draw_networkx_labels(G, positions, labels = labels)

/home/dmitriy/MyEnv/other/skoltech/intro-to-blockchain/env/lib/python3.5/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


{0: Text(0, 4, '0'),
 1: Text(-3, 3, '1'),
 2: Text(0, 3, '2'),
 3: Text(3, 3, '3'),
 4: Text(-4, 2, '4'),
 5: Text(-3, 2, '5'),
 6: Text(-2, 2, '6'),
 7: Text(-1, 2, '7'),
 8: Text(0, 2, '8'),
 9: Text(1, 2, '9'),
 10: Text(2, 2, '10'),
 11: Text(3, 2, '11'),
 12: Text(4, 2, '12'),
 13: Text(-5, 1, 'tx1'),
 14: Text(-4, 1, 'tx2'),
 15: Text(-3, 1, 'tx3'),
 16: Text(-2, 1, 'tx4'),
 17: Text(-1, 1, 'tx5'),
 18: Text(0, 1, 'tx6'),
 19: Text(1, 1, 'tx7'),
 20: Text(2, 1, 'tx8'),
 21: Text(3, 1, 'tx9')}

In [263]:
G = nx.Graph()
positions = {}
coordinates = [
    [0, 4],
    [-2, 3],
    [2, 3],
    [-3, 2],
    [-1, 2],
    [1, 2],
    [3, 2],
    [-4, 1],
    [-3, 1],
    [-2, 1],
    [-1, 1],
    [0, 1],
    [1, 1],
    [2, 1],
    [3, 1],
    [-4, 0],
    [-3, 0],
    [-2, 0],
    [-1, 0],
    [0, 0],
    [1, 0],
    [2, 0],
    [3, 0],
    [4, 0],
    [5, 0],
    [6, 0],
    [7, 0],
    [8, 0],
    [9, 0],
    [10, 0],
    [11, 0],
    [0, -2],
    [1, -2],
    [2, -2],
    [3, -2],
    [4, -2],
    [5, -2],
    [6, -2],
    [7, -2],
    [8, -2]
]
parents = [0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
for index in range(40):
    G.add_node(index)
    G.add_edge(index, parents[index])
    positions[index] = coordinates[index]
nx.draw(G, positions, node_size = 1000)
labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: '7',
    8: '8',
    9: '9',
    10: '10',
    11: '11',
    12: '12',
    13: '13',
    14: '14',
    15: '15',
    16: '16',
    17: '17',
    18: '18',
    19: '19',
    20: '20',
    21: '21',
    22: '22',
    23: '23',
    24: '24',
    25: '25',
    26: '26',
    27: '27',
    28: '28',
    29: '29',
    30: '30',
    31: 'tx1',
    32: 'tx2',
    33: 'tx3',
    34: 'tx4',
    35: 'tx5',
    36: 'tx6',
    37: 'tx7',
    38: 'tx8',
    39: 'tx9',
}
nx.draw_networkx_labels(G, positions, labels = labels)

{0: Text(0, 4, '0'),
 1: Text(-2, 3, '1'),
 2: Text(2, 3, '2'),
 3: Text(-3, 2, '3'),
 4: Text(-1, 2, '4'),
 5: Text(1, 2, '5'),
 6: Text(3, 2, '6'),
 7: Text(-4, 1, '7'),
 8: Text(-3, 1, '8'),
 9: Text(-2, 1, '9'),
 10: Text(-1, 1, '10'),
 11: Text(0, 1, '11'),
 12: Text(1, 1, '12'),
 13: Text(2, 1, '13'),
 14: Text(3, 1, '14'),
 15: Text(-4, 0, '15'),
 16: Text(-3, 0, '16'),
 17: Text(-2, 0, '17'),
 18: Text(-1, 0, '18'),
 19: Text(0, 0, '19'),
 20: Text(1, 0, '20'),
 21: Text(2, 0, '21'),
 22: Text(3, 0, '22'),
 23: Text(4, 0, '23'),
 24: Text(5, 0, '24'),
 25: Text(6, 0, '25'),
 26: Text(7, 0, '26'),
 27: Text(8, 0, '27'),
 28: Text(9, 0, '28'),
 29: Text(10, 0, '29'),
 30: Text(11, 0, '30'),
 31: Text(0, -2, 'tx1'),
 32: Text(1, -2, 'tx2'),
 33: Text(2, -2, 'tx3'),
 34: Text(3, -2, 'tx4'),
 35: Text(4, -2, 'tx5'),
 36: Text(5, -2, 'tx6'),
 37: Text(6, -2, 'tx7'),
 38: Text(7, -2, 'tx8'),
 39: Text(8, -2, 'tx9')}

Note that **tx9** will be copied to nodes on the right side when we'll start to calculate hashes

6.What is the optimum tree fanout for signing first 4 values in the set up of previous item? (2 points)

We definetely have to calculate 4 hashes of the values. Also we'll have to make hash calculation for some concatenated value from the other parts. So we can't make less than 5 hash calculation operations.

The answer is **9**. In this case we'll have to calculate hash exactly 5 times. 
$$bitcoinhash(bitcoinhash(tx1)+bitcoinhash(tx2) + bitcoinhash(tx3)+bitcoinhash(tx4) + hash_{15} + hash_{16} + hash_{17} + hash_{18} + hash_{19})$$